In [1]:
# https://medium.com/@myworldsharma.jay/key-frame-extraction-from-video-9445564eb8ed
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np
import os
import math

In [2]:
def create_folders(file_name_only_no_extension):    
    try:
        intermediate_location="../intermediate/"
        if not os.path.isdir(intermediate_location):
            os.mkdir(intermediate_location)
        resized_location=intermediate_location+"resized/"
        if not os.path.isdir(resized_location):
            os.mkdir(resized_location)
        
        

            
        data_dir=intermediate_location+"data/"
        if not os.path.isdir(data_dir):
            os.mkdir(data_dir)
        data_dir=data_dir+file_name_only_no_extension+"/"  
        if not os.path.isdir(data_dir):
            os.mkdir(data_dir)    
            

        
            
        kframes_location=intermediate_location+"keyframes/"
        if not os.path.exists(kframes_location):
            os.makedirs(kframes_location)
        kframes_location+=file_name_only_no_extension+"/"
        if not os.path.exists(kframes_location):
            os.makedirs(kframes_location)        
    except OSError:
        print("Error cant make directories")
        
    return resized_location,data_dir,kframes_location
        
def resize_video(source_video_location,destination):
    
    new_dim=(32,24)

    cap = cv2.VideoCapture(source_video_location)

    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(destination, fourcc, 25.0, new_dim)


    while True:
        ret, frame = cap.read()
        if ret == True:
            b = cv2.resize(frame,new_dim,fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
            out.write(b)
        else:
            break

    cap.release()
    out.release()
#     cv2.destroyAllWindows()    


    
def extract_all_frames_histogram(video_location,all_frames_location):

    cap = cv2.VideoCapture(video_location)
    fps = cap.get(cv2.CAP_PROP_FPS)
    print("Frames per second using video.get(cv2.CV_CAP_PROP_FPS): {0}".format(fps))
    frame_list_norm = []
    hist_list=[]
    print("Start")


    cframe = 0
    while(True):

        ret, frame= cap.read()
        if not ret:
            break
        frame_arr=np.array(frame)
        frame_arr_norm=(frame_arr-0)/255
        frame_list_norm.append(frame_arr_norm)
        hist = cv2.calcHist([frame],[0,1,2],None,[8,8,8],[0,256,0,256,0,256])
        hist = cv2.normalize(hist,None).flatten()
        hist_list.append(hist)
        name = all_frames_location +str(cframe) + '.jpg'
        cv2.imwrite(name,frame)

        cframe += 1

    cap.release()
#     cv2.destroyAllWindows()
    print("Frame extraction complete")     
    return hist_list



# whenever there is a 1
# make k frames before and after it also 1

def make_one(list_,center,window,to_keep_new):
#     print("center is at",center,"from",center-window,"to",center+window)
    for i in range(center-window,center+window):
        if i>0 and i<len(list_):
            to_keep_new[i]=1
    return to_keep_new



def generate_keep_list(hist_list):


    hist_list=np.array(hist_list)
#     print(hist_list.shape)

    diff_hist_list=[]

    for i in range(hist_list.shape[0]-1):
        hist1=hist_list[i]
        hist2=hist_list[i+1]
        d = abs(cv2.compareHist(hist1, hist2, cv2.HISTCMP_INTERSECT))
#         print(d)
        diff_hist_list.append(d)

    diff_hist_list=np.array(diff_hist_list)
#     print(diff_hist_list.shape)
    sd=math.sqrt(np.var(diff_hist_list))



    differences=[]
    left_image_index=0
    for right_image_index in range(1,diff_hist_list.shape[0]):
        # to check how much right image is similar to the left image
        d1=diff_hist_list[left_image_index]
        d2=diff_hist_list[right_image_index]
        diff_diff = abs(d1-d2)
        differences.append(diff_diff)
        left_image_index=right_image_index


    sd_diff=math.sqrt(np.var(differences))    


    to_keep=[1]

    left_image_index=0
    for right_image_index in range(1,diff_hist_list.shape[0]):
        # to check how much right image is similar to the left image
        d1=diff_hist_list[left_image_index]
        d2=diff_hist_list[right_image_index]
        diff_diff = abs(d1-d2)
#         print(left_image_index,right_image_index,diff_diff,sd_diff)
        if diff_diff>sd_diff:
            to_keep.append(1)
        else:
            to_keep.append(0)
        left_image_index=right_image_index



    wind=1
    to_keep_new=[0 for i in range(len(to_keep))]
    for i in range(len(to_keep)):
        if to_keep[i]==1:
#             print("1 at ",i,to_keep[i])
            to_keep_new=make_one(to_keep,i,wind,to_keep_new)                

    return to_keep_new

def store_keyframes_using_keep_list(video_location,kframes_location,keep_list):
    cap = cv2.VideoCapture(video_location)    
    fps = cap.get(cv2.CAP_PROP_FPS)    
    cframe = 0
    while(True):
        ret, frame= cap.read()
        if not ret:
            break        
        if keep_list[cframe]==1:
            name = kframes_location + str(cframe) + '.jpg'
#             print("creating" +name)
            cv2.imwrite(name,frame)
        cframe += 1
        if cframe>len(keep_list)-1:
            break

    cap.release()
#     cv2.destroyAllWindows()
    print("Data creation complete")          


def make_keyframes(source_video_location):
    
    file_name_only=source_video_location.split("/")[-1]
    file_name_only_no_extension=file_name_only.split(".")[0]

    print(file_name_only,file_name_only_no_extension)
    # create all the requisite folders
    resized_location,all_frames_location,kframes_location=create_folders(file_name_only_no_extension)
    
    # can we put a check to see if the kframes_location already has files in it
    if len(os.listdir(kframes_location))>0:
        print("Keyframes already made for this file")
        return kframes_location
    
    print(resized_location,all_frames_location,kframes_location)
    
    resized_location_destination=resized_location+file_name_only
    
    # resize the video to be same size
    resize_video(source_video_location,resized_location_destination)
    
    # extract all frames, store in data folder and generate histogram
    hist_list=extract_all_frames_histogram(resized_location_destination,all_frames_location)
    
    
    to_keep_new=generate_keep_list(hist_list)
    
    store_keyframes_using_keep_list(resized_location_destination,kframes_location,to_keep_new)
    print("All keyframes at",kframes_location)
    return kframes_location

### two references

In [3]:
path="/ddn/gfxhome/asislam25/projects/other_misc_projects/video_copy/data/references/"
file_names=os.listdir(path)
klocs=[]
for file_name in file_names:
    file_name=path+file_name
    kloc=make_keyframes(file_name)
    klocs.append(kloc)

c05_202102080735.mp4 c05_202102080735
../intermediate/resized/ ../intermediate/data/c05_202102080735/ ../intermediate/keyframes/c05_202102080735/
Frames per second using video.get(cv2.CV_CAP_PROP_FPS): 25.0
Start
Frame extraction complete
Data creation complete
All keyframes at ../intermediate/keyframes/c05_202102080735/
c03_202102071755.mp4 c03_202102071755
Keyframes already made for this file
c05_202101300845.mp4 c05_202101300845
../intermediate/resized/ ../intermediate/data/c05_202101300845/ ../intermediate/keyframes/c05_202101300845/
Frames per second using video.get(cv2.CV_CAP_PROP_FPS): 25.0
Start
Frame extraction complete
Data creation complete
All keyframes at ../intermediate/keyframes/c05_202101300845/
c01_202102140630.mp4 c01_202102140630
../intermediate/resized/ ../intermediate/data/c01_202102140630/ ../intermediate/keyframes/c01_202102140630/
Frames per second using video.get(cv2.CV_CAP_PROP_FPS): 25.0
Start
Frame extraction complete
Data creation complete
All keyframes at 

In [4]:
# file_name="/Users/ashhadulislam/projects/other_misc/video_copy/data/references/c01_202101300630.mp4"
# kloc=make_keyframes(file_name)
# print("Check keyframes at",kloc)

In [5]:
# file_name="/Users/ashhadulislam/projects/other_misc/video_copy/data/references/c01_202101300810.mp4"
# kloc=make_keyframes(file_name)
# print("Check keyframes at",kloc)

### Three archives

In [6]:
# file_name="/Users/ashhadulislam/projects/other_misc/video_copy/data/setC/positive-c.zip/c01_20210130063213_C2.mp4"
# kloc=make_keyframes(file_name)
# print("Check keyframes at",kloc)

In [7]:
# file_name="/Users/ashhadulislam/projects/other_misc/video_copy/data/setC/positive-c.zip/c01_20210130081629_C3.mp4"
# kloc=make_keyframes(file_name)
# print("Check keyframes at",kloc)

In [8]:
# file_name="/Users/ashhadulislam/projects/other_misc/video_copy/data/setC/positive-c.zip/c01_20210130115216_C3.mp4"
# kloc=make_keyframes(file_name)
# print("Check keyframes at",kloc)

In [9]:
path="/ddn/gfxhome/asislam25/projects/other_misc_projects/video_copy/data/setC/positive-c.zip/"
file_names=os.listdir(path)
num_files=len(file_names)
klocs_arch=[]
count=0
for file_name in file_names:
    count+=1
    print(count,"/",num_files)
    file_name=path+file_name
    kloc=make_keyframes(file_name)
    klocs_arch.append(kloc)

1 / 50
c09_20210210135939_C2.mp4 c09_20210210135939_C2
Keyframes already made for this file
2 / 50
c01_20210203062229_C4.mp4 c01_20210203062229_C4
Keyframes already made for this file
3 / 50
c01_20210205062047_C4.mp4 c01_20210205062047_C4
Keyframes already made for this file
4 / 50
c09_20210222174848_C1.mp4 c09_20210222174848_C1
../intermediate/resized/ ../intermediate/data/c09_20210222174848_C1/ ../intermediate/keyframes/c09_20210222174848_C1/
Frames per second using video.get(cv2.CV_CAP_PROP_FPS): 25.0
Start
Frame extraction complete
Data creation complete
All keyframes at ../intermediate/keyframes/c09_20210222174848_C1/
5 / 50
c09_20210227162920_C1.mp4 c09_20210227162920_C1
../intermediate/resized/ ../intermediate/data/c09_20210227162920_C1/ ../intermediate/keyframes/c09_20210227162920_C1/
Frames per second using video.get(cv2.CV_CAP_PROP_FPS): 25.0
Start
Frame extraction complete
Data creation complete
All keyframes at ../intermediate/keyframes/c09_20210227211344_C1/
13 / 50
c09_20

Frames per second using video.get(cv2.CV_CAP_PROP_FPS): 25.0
Start
Frame extraction complete
Data creation complete
All keyframes at ../intermediate/keyframes/c09_20210228085036_C3/
48 / 50
c09_20210221125002_C2.mp4 c09_20210221125002_C2
../intermediate/resized/ ../intermediate/data/c09_20210221125002_C2/ ../intermediate/keyframes/c09_20210221125002_C2/
Frames per second using video.get(cv2.CV_CAP_PROP_FPS): 25.0
Start
Frame extraction complete
Data creation complete
All keyframes at ../intermediate/keyframes/c09_20210221125002_C2/
49 / 50
c09_20210221172350_C4.mp4 c09_20210221172350_C4
../intermediate/resized/ ../intermediate/data/c09_20210221172350_C4/ ../intermediate/keyframes/c09_20210221172350_C4/
Frames per second using video.get(cv2.CV_CAP_PROP_FPS): 25.0
Start
Frame extraction complete
Data creation complete
All keyframes at ../intermediate/keyframes/c09_20210221172350_C4/
50 / 50
c01_20210130194220_C2.mp4 c01_20210130194220_C2
Keyframes already made for this file


In [10]:
klocs_arch

['../intermediate/keyframes/c09_20210210135939_C2/',
 '../intermediate/keyframes/c01_20210203062229_C4/',
 '../intermediate/keyframes/c01_20210205062047_C4/',
 '../intermediate/keyframes/c09_20210222174848_C1/',
 '../intermediate/keyframes/c09_20210227162920_C1/',
 '../intermediate/keyframes/c01_20210210081914_C3/',
 '../intermediate/keyframes/c01_20210131062134_C1/',
 '../intermediate/keyframes/c09_20210224174314_C2/',
 '../intermediate/keyframes/c05_20210203202303_C1/',
 '../intermediate/keyframes/c01_20210130210232_C4/',
 '../intermediate/keyframes/c09_20210221115250_C2/',
 '../intermediate/keyframes/c09_20210227211344_C1/',
 '../intermediate/keyframes/c09_20210225175756_C2/',
 '../intermediate/keyframes/c01_20210130124435_C1/',
 '../intermediate/keyframes/c01_20210131205546_C2/',
 '../intermediate/keyframes/c09_20210228105604_C4/',
 '../intermediate/keyframes/c09_20210228124351_C2/',
 '../intermediate/keyframes/c01_20210130063213_C2/',
 '../intermediate/keyframes/c01_20210202125721